In [1]:
from opcua import Client
import time

def extract_trend_node_names(endpoint, probe_node_id="ns=2;s=Local.iCIR.Probe1.Trends"):
    """
    Connects to an OPC UA server and extracts the names of child nodes under the specified Probe1 Trends node.
    
    Args:
        endpoint (str): OPC UA server endpoint (e.g., "opc.tcp://localhost:62552/iCOpcUaServer")
        probe_node_id (str): NodeId of the Trends node for Probe1
    
    Returns:
        List of trend node display names (str)
    """
    client = Client(endpoint)
    trend_names = []

    try:
        print(f"Connecting to OPC UA server at {endpoint}...")
        client.connect()
        print("Connected.")

        trend_node = client.get_node(probe_node_id)
        children = trend_node.get_children()

        print(f"Found {len(children)} children in Probe1.Trends:")
        for child in children:
            try:
                name = child.get_display_name().Text
                print(f" - {name}")
                trend_names.append(name)
            except Exception as e:
                print(f"Could not get name for node {child}: {e}")

    except Exception as e:
        print(f"Error: {e}")
    finally:
        client.disconnect()
        print("Disconnected.")

    return trend_names


endpoint = "opc.tcp://localhost:62552/iCOpcUaServer"
trend_names = extract_trend_node_names(endpoint)

Connecting to OPC UA server at opc.tcp://localhost:62552/iCOpcUaServer...
Connected.
Found 7 children in Probe1.Trends:
 - Probe Temp
 - cage
 - Peak at 1155 cm-1
 - Peak at 1680 cm-1
 - Peak at 1088 cm-1
 - Peak at 1649 cm-1
 - THF Peak at 1264 cm-1
Disconnected.


In [1]:
import os

logs_dir = "logs"
print(f"Logs directory exists? {os.path.exists(logs_dir)}")
print(f"Is writable? {os.access(logs_dir, os.W_OK)}")

if not os.path.exists(logs_dir):
    print("Creating logs directory...")
    os.makedirs(logs_dir)

if not os.access(logs_dir, os.W_OK):
    print("Logs directory is not writable! Please fix permissions.")
else:
    print("Logs directory is writable.")


Logs directory exists? True
Is writable? True
Logs directory is writable.


In [2]:
import os
import sqlite3

db_path = "ReactIR.db"

print(f"Database file exists? {os.path.exists(db_path)}")
print(f"Database file writable? {os.access(db_path, os.W_OK)}")

conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# List tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables in DB:", tables)

# Replace 'TrendTableName' below with the actual table your create_new_trend() writes to:
trend_table_name = "trends"  # Change as needed!

if (trend_table_name,) in tables:
    cursor.execute(f"PRAGMA table_info({trend_table_name});")
    columns = cursor.fetchall()
    print(f"Columns in '{trend_table_name}' table:", columns)
else:
    print(f"Table '{trend_table_name}' does not exist in the database.")

conn.close()


Database file exists? True
Database file writable? True
Tables in DB: [('Users',), ('Projects',), ('Experiments',), ('Documents',), ('Probes',), ('Samples',), ('Spectra',), ('Reagents',), ('Trends',), ('sqlite_sequence',), ('ProbeTempSamples',), ('PeakSamples',)]
Table 'trends' does not exist in the database.


In [4]:
from db_utils import setup_database

db_path = "ReactIR.db"
setup_database(db_path)
print("✅ Database schema initialized or verified.")


Error logger has not been configured with a path.
✅ Database schema initialized or verified.


In [1]:
import sqlite3

# Replace this with your actual database file
db_path = 'ReactIR.db'

# Connect to the SQLite database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Get all table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Print each table's contents
for table_name in tables:
    table_name = table_name[0]
    print(f"\n--- Contents of table: {table_name} ---")
    
    # Get all data from the table
    cursor.execute(f"SELECT * FROM {table_name}")
    rows = cursor.fetchall()

    # Get column names
    column_names = [description[0] for description in cursor.description]
    print(" | ".join(column_names))
    
    for row in rows:
        print(" | ".join(str(cell) for cell in row))

# Clean up
conn.close()



--- Contents of table: Users ---
UserID | Username
1 | FlowIR

--- Contents of table: Projects ---
ProjectID | Name | UserID
1 | Project Alpha | 1

--- Contents of table: Experiments ---
ExperimentID | Name | ProjectID
1 | Exp 001 | 1

--- Contents of table: Documents ---
DocumentID | Name | ExperimentID
1 | Doc 1 | 1
2 | Unknown_Experiment | None
3 | Unknown_Experiment | None
4 | Unknown_Experiment | None
5 | Unknown_Experiment | None
6 | Unknown_Experiment | None
7 | Unknown_Experiment | None
8 | Unknown_Experiment | None

--- Contents of table: Probes ---
ProbeID | Description | DocumentID | LatestTemperatureCelsius | LatestTemperatureTime
1 | Probe A Description | 1 | 23.5 | 2025-07-03 14:00:00
2 | No description | -1 | None | None
3 | No description | -1 | None | None
4 | No description | -1 | None | None
5 | No description | -1 | None | None
6 | No description | -1 | None | None
7 | No description | -1 | None | None
8 | No description | -1 | None | None
9 | No description | -1 |

In [2]:
import sqlite3
import pandas as pd

# Path to your SQLite database
db_path = 'ReactIR.db'

# Connect to database
conn = sqlite3.connect(db_path)

# Get all table names
table_query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql(table_query, conn)

# Print each table as a DataFrame
for table_name in tables['name']:
    print(f"\n--- Table: {table_name} ---")
    df = pd.read_sql(f"SELECT * FROM {table_name}", conn)
    print(df)

# Close connection
conn.close()




--- Table: Users ---
   UserID Username
0       1   FlowIR

--- Table: Projects ---
   ProjectID           Name  UserID
0          1  Project Alpha       1

--- Table: Experiments ---
   ExperimentID     Name  ProjectID
0             1  Exp 001          1

--- Table: Documents ---
   DocumentID                Name  ExperimentID
0           1               Doc 1           1.0
1           2  Unknown_Experiment           NaN
2           3  Unknown_Experiment           NaN
3           4  Unknown_Experiment           NaN
4           5  Unknown_Experiment           NaN
5           6  Unknown_Experiment           NaN
6           7  Unknown_Experiment           NaN
7           8  Unknown_Experiment           NaN

--- Table: Probes ---
     ProbeID          Description  DocumentID  LatestTemperatureCelsius  \
0          1  Probe A Description           1                      23.5   
1          2       No description          -1                       NaN   
2          3       No description    

In [3]:
import sqlite3

conn = sqlite3.connect("ReactIR.db")
cursor = conn.cursor()

try:
    cursor.execute("ALTER TABLE Documents ADD COLUMN ErrorLogPath TEXT;")
    conn.commit()
    print("Column added successfully.")
except sqlite3.OperationalError as e:
    print(f"Error: {e} (maybe column already exists?)")
finally:
    conn.close()


Column added successfully.
